# Generator
- a type of iterator/are iterators, they implement the iterator protocol.
- Generator are ingerently laxy iterators (can be infinite)
- Generator are iterators, and can be used in the same way (for loops, comprehension)
- it can be exhausted, cannot be 'restored'
- This leads to a bug, if we try to iterate twice over a generator.
-
### Generator Function
- Generator factories-> they return a Generator when called
- a function that uses the yield statement
- generator function always returns an iterator.
-
### Generator expression:
- uses comprehension syntax
- a more concise way of creating Generators
- like list comprehension, useful for simple situations.

## Yield
- The yield keyword does exactly what we want:
- It emits a value
- the function is effectively suspended(but it retains it current state)
- calling next on the function resumes running the function right after the yield statement
- if function returns somethings instead if yielding -> StopIteration exception.

In [18]:
def song():
    print('line 1')
    yield 'I am a lumberjack and i am OK'
    print('line 2')
    yield 'i sleep all night and i work all day'

lines = song()
type(lines)
# calling a function, returns a genertaor object
# the returned generator is executed by calling next()
# the function body will execute untill it encourters a yield statement.

generator

In [19]:
line = next(lines)
line

line 1


'I am a lumberjack and i am OK'

In [20]:
line = next(lines)
line

line 2


'i sleep all night and i work all day'

In [21]:
line = next(lines)
line

StopIteration: 

In [19]:
# Example using iterator
import math
class FactIter:
    def __init__(self,n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i>=self.n:
            raise StopIteration
        else:
            result = math.factorial(self.i)
            self.i +=1
            return result

fact_iter = FactIter(5)
list(fact_iter)

[1, 1, 2, 6, 24]

In [20]:
# using closure
import math
def fact():
    i = 0
    def inner():
        nonlocal i
        result = math.factorial(i)
        i +=1
        return result
    return inner

fact_inter = iter(fact(),math.factorial(5))

list(fact_inter)

[1, 1, 2, 6, 24]

In [21]:
# using generator
def factorails(n):
    for i in range(n):
        yield math.factorial(i)

fact_gen = factorails(5)
list(fact_gen)

[1, 1, 2, 6, 24]

# Example - Fibonacci Sequence

In [4]:
# we can use recursive
from timeit import timeit

def fib_recursive(n):
    if n<=1:
        return 1
    else:
        return fib_recursive(n-1)+fib_recursive(n-2)

print([fib_recursive(i) for i in range(8)])
# it is fine if the number is small, what if the number of large output-> it is slow.
timeit('fib_recursive(30)',globals=globals(),number=10)

[1, 1, 2, 3, 5, 8, 13, 21]


4.314233899999863

In [5]:
# we can use memoization.
from functools import lru_cache

@lru_cache
def fib_recursive_lru(n):
    if n<=1:
        return 1
    else:
        return fib_recursive_lru(n-1)+fib_recursive_lru(n-2)

print([fib_recursive_lru(i) for i in range(8)])
print(timeit('fib_recursive_lru(30)',globals=globals(),number=10))
# it now preety fast, but it still going to be prob. the depth of python. using lot of memory is not good.
fib_recursive_lru(2000)
# The maximum recursion depth exceeded.

[1, 1, 2, 3, 5, 8, 13, 21]
1.950000000761065e-05


RecursionError: maximum recursion depth exceeded in comparison

In [8]:
# Then we can use loops -> for
def fib_loop(n):
    f1 = 1
    f2 = 1
    for i in range(n):
        f1,f2 = f2,f1+f2
    return f1

print([fib_loop(i) for i in range(8)])
print(timeit('fib_loop(30)',globals=globals(),number=10))

[1, 1, 2, 3, 5, 8, 13, 21]


NameError: name 'timeit' is not defined

In [9]:
# let use iterator in loop
class FibIter:
    def __init__(self,n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i>=self.n:
            raise StopIteration
        else:
            result = fib_loop(self.i)
            self.i+=1
            return result
fib_loop_class = FibIter(9)
print([num for num in fib_loop_class])
# now here, we have to write a class, just to print a fibonacci series. we can use generator to perform the same task, easier and faster.

[1, 1, 2, 3, 5, 8, 13, 21, 34]


In [11]:
# using yield
def fib_gen(n):
    i,j = 0,1
    for _ in range(n):
        i,j = j,i+j
        yield i

In [12]:
f= fib_gen(9)
[f for f in f]

[1, 1, 2, 3, 5, 8, 13, 21, 34]

In [22]:
from timeit import timeit
print(timeit('fib_loop(5000)',globals=globals(),number=10))
print(timeit('list(FibIter(5000))',globals=globals(),number=10))
print(timeit('fib_gen(5000)',globals=globals(),number=10))

0.00915199999997185
17.034362200000032
4.800000056093268e-06


In [ ]:
# Here loop is faster, but, we need to call all values.
# using class gives __iter__, but it is slower
# using generator to yield or __iter__ is fast and convinent.

# Making a Iterable from a Generator
- it can be exhausted, cannot be 'restored'
- This leads to a bug, if we try to iterate twice over a generator.

In [31]:
# eg:
def square(n):
    for i in range(n):
        yield i**2

sq = square(5)
list(sq)

[0, 1, 4, 9, 16]

In [27]:
# This can lead us to unexpected behaviour sometimes.
sq = square(5)

# what if we want to enumerate the sq.
enum1 = enumerate(sq) # enumerate is lazy-> hasn't iterated through 'sq' yet.
print(next(sq))
print(next(sq))
list(enum1) # here enum1 is called here so, enumerator does know, that sq, starts with 2, with enumerate with 0.

0
1


[(0, 4), (1, 9), (2, 16)]

In [39]:
# Making an iterator
class Squares:
    def __init__(self,n):
        self.n = n
    
    def __iter__(self):
        return square(self.n) # new instance of the generator

sq = Squares(5)
print(list(sq))
print(list(sq))

[0, 1, 4, 9, 16]
[0, 1, 4, 9, 16]


# Example : Card Deck

In [42]:
from collections import namedtuple

Card = namedtuple('Card','rand suit')

SUITS = ('Spades','Hearts','Diamonds','Clubs')
RANKS = tuple(range(2,11))+tuple('JQKA')

In [48]:
"""
suit_index = card_index // len(RANKS)
rand_index = card_index % len(RANKS)
"""

def card_gen():
    for i in range(len(SUITS)*len(RANKS)):
        suit = SUITS[i//len(RANKS)]
        rank = RANKS[i%len(RANKS)]
        card = Card(rank,suit)
        yield card

card1 = card_gen()
list(card1)[0:12]

[Card(rand=2, suit='Spades'),
 Card(rand=3, suit='Spades'),
 Card(rand=4, suit='Spades'),
 Card(rand=5, suit='Spades'),
 Card(rand=6, suit='Spades'),
 Card(rand=7, suit='Spades'),
 Card(rand=8, suit='Spades'),
 Card(rand=9, suit='Spades'),
 Card(rand=10, suit='Spades'),
 Card(rand='J', suit='Spades'),
 Card(rand='Q', suit='Spades'),
 Card(rand='K', suit='Spades')]

In [47]:
def card_gen():
    for suit in SUITS:
        for rank in RANKS:
            yield Card(rank,suit)

card1 = card_gen()
list(card1)[0:12]

[Card(rand=2, suit='Spades'),
 Card(rand=3, suit='Spades'),
 Card(rand=4, suit='Spades'),
 Card(rand=5, suit='Spades'),
 Card(rand=6, suit='Spades'),
 Card(rand=7, suit='Spades'),
 Card(rand=8, suit='Spades'),
 Card(rand=9, suit='Spades'),
 Card(rand=10, suit='Spades'),
 Card(rand='J', suit='Spades'),
 Card(rand='Q', suit='Spades'),
 Card(rand='K', suit='Spades')]

In [62]:
class CardDeck:
    SUITS = ('Spaces','Hearts','Diamonds','Clubs')
    RANKS = tuple(range(2,11))+tuple('JQKA')

    def __iter__(self):
        return CardDeck.card_gen(self)
    
    def __reversed__(self):
        return CardDeck.reversed_card_gen(self)

    @staticmethod
    def card_gen(self):
        for suit in self.SUITS:
            for rank in self.RANKS:
                yield Card(rank,suit)

    @staticmethod
    def reversed_card_gen(self):
        for suit in self.SUITS[::-1]:
            for rank in self.RANKS[::-1]:
                yield Card(rank,suit)

In [64]:
deck = CardDeck()
list(deck)[0:11]

[Card(rand=2, suit='Spaces'),
 Card(rand=3, suit='Spaces'),
 Card(rand=4, suit='Spaces'),
 Card(rand=5, suit='Spaces'),
 Card(rand=6, suit='Spaces'),
 Card(rand=7, suit='Spaces'),
 Card(rand=8, suit='Spaces'),
 Card(rand=9, suit='Spaces'),
 Card(rand=10, suit='Spaces'),
 Card(rand='J', suit='Spaces'),
 Card(rand='Q', suit='Spaces')]

In [60]:
list(deck)[0:11] # now we can call as many time we want.

[Card(rand=2, suit='Spaces'),
 Card(rand=3, suit='Spaces'),
 Card(rand=4, suit='Spaces'),
 Card(rand=5, suit='Spaces'),
 Card(rand=6, suit='Spaces'),
 Card(rand=7, suit='Spaces'),
 Card(rand=8, suit='Spaces'),
 Card(rand=9, suit='Spaces'),
 Card(rand=10, suit='Spaces'),
 Card(rand='J', suit='Spaces'),
 Card(rand='Q', suit='Spaces')]

In [66]:
list(reversed(deck))[0:11]

[Card(rand='A', suit='Clubs'),
 Card(rand='K', suit='Clubs'),
 Card(rand='Q', suit='Clubs'),
 Card(rand='J', suit='Clubs'),
 Card(rand=10, suit='Clubs'),
 Card(rand=9, suit='Clubs'),
 Card(rand=8, suit='Clubs'),
 Card(rand=7, suit='Clubs'),
 Card(rand=6, suit='Clubs'),
 Card(rand=5, suit='Clubs'),
 Card(rand=4, suit='Clubs')]

# Generator Expressions: and Performance.
- it uses the same syntax as list comprehensions syntax-> including nesting, if
- instead of [], we Use ()
- instead of returning list, it return generator
-
- instead if evaluation is eagar, it evaluation is lazy
- instead of iterable, it is iterator.
- instead of creating object right away, they are delayed untill requested.
- list comprehensions takes longer to create/return the list,so iteartion is faster(object already created)
- generator is createdretuned immediately. iteartion is slower(object need to be created)
- instead of loadint entire collection to memory, it only load only a single item.
    - #### generator dont take all memory at a time, it uses memory when they are called.
    - if we have a huge file, and done list comprehensions, it will aquire more memory. in this type of context, it better to use generators.
-
- ## Performance
    - If you iterate through all the elements -> time Performance is about to same
    - but, if you dont iterate through all the elements-> generator is more efficient.
    - if we, just have to access certain no of transaction from database, instead of loading whole transaction.

In [71]:
# list comprehension
l = [ i**2 for i in range(5)]
print(type(l))
l

<class 'list'>


[0, 1, 4, 9, 16]

In [76]:
# generator
l = ( i**2 for i in range(5))
print(type(l))
list(l)

<class 'generator'>


[0, 1, 4, 9, 16]

In [78]:
# multiple list using list comprehension
start = 1
stop = 10
multi_list = [[i*j for i in range(start,stop+1)] for j in range(start,stop+1)]
multi_list

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
 [3, 6, 9, 12, 15, 18, 21, 24, 27, 30],
 [4, 8, 12, 16, 20, 24, 28, 32, 36, 40],
 [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
 [6, 12, 18, 24, 30, 36, 42, 48, 54, 60],
 [7, 14, 21, 28, 35, 42, 49, 56, 63, 70],
 [8, 16, 24, 32, 40, 48, 56, 64, 72, 80],
 [9, 18, 27, 36, 45, 54, 63, 72, 81, 90],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]]

In [91]:
# multiple list using generator comprehension
start = 1
stop = 10
multi_list = ((i*j for i in range(start,stop+1)) for j in range(start,stop+1))
print(multi_list)

for i in multi_list:
    print(tuple(i))

<generator object <genexpr> at 0x000001A0A71B4580>
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
(2, 4, 6, 8, 10, 12, 14, 16, 18, 20)
(3, 6, 9, 12, 15, 18, 21, 24, 27, 30)
(4, 8, 12, 16, 20, 24, 28, 32, 36, 40)
(5, 10, 15, 20, 25, 30, 35, 40, 45, 50)
(6, 12, 18, 24, 30, 36, 42, 48, 54, 60)
(7, 14, 21, 28, 35, 42, 49, 56, 63, 70)
(8, 16, 24, 32, 40, 48, 56, 64, 72, 80)
(9, 18, 27, 36, 45, 54, 63, 72, 81, 90)
(10, 20, 30, 40, 50, 60, 70, 80, 90, 100)


# Yield From

In [97]:
def matrix(n):
    gen = ((i*j for j in range(1,n+1)) for i in range(1,n+1))
    return gen

In [98]:
def matrix_iterator(n):
    for row in matrix(n):
        for item in row:
            yield item

for item in matrix_iterator(3):
    print(item)

1
2
3
2
4
6
3
6
9


In [99]:
#using yield from

def matrix_iterator(n):
    for row in matrix(n):
        yield from row

for item in matrix_iterator(3):
    print(item)

1
2
3
2
4
6
3
6
9
